In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive
/content/gdrive/My Drive


In [ ]:
import pandas as pd
data = pd.read_excel(r'Time series project/Data/ATM/NN5.xlsx')

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

# Set 'Date' as the index for easy time series manipulation
data.set_index('Date', inplace=True)
data.replace(0, pd.NA, inplace=True)  # Replace zeros with NaN
data.fillna(data.mean(), inplace=True)  # Fill NaN with the column mean

<ipython-input-3-b672e48e23b2>:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(data.mean(), inplace=True)  # Fill NaN with the column mean


# Untuned

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import time

# Assuming the data is already loaded as 'data'
n_test = 30
n_lag = 100  # Rolling window size

# Function to create rolling windows
def create_rolling_window(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

# Placeholder for results
results = []

# Set the same hyperparameters for all models
n_epochs = 100
batch_size = 32
n_filters = 32  # Number of filters in Conv1D
kernel_size = 3  # Size of the convolutional kernel
input_shape = (n_lag, 1)  # CNN input is (n_lag, 1), where 1 is the number of features

# Scaling the data using MinMaxScaler (0 to 1 range)
scaler = MinMaxScaler()

# Loop over each column (excluding the 'Date' column)
for col in data.columns:
    print(f'Training CNN model for column: {col}')

    # Prepare the data for the current column
    series = data[col].values.reshape(-1, 1)  # Reshape for scaling (1 feature)

    # Scale the series
    series_scaled = scaler.fit_transform(series)

    # Create rolling windows
    X, y = create_rolling_window(series_scaled, n_lag)

    # Split into train and test
    X_train, X_test = X[:-n_test], X[-n_test:]
    y_train, y_test = y[:-n_test], y[-n_test:]

    # Reshape for CNN, needs 3D shape (samples, timesteps, features)
    X_train = X_train.reshape((X_train.shape[0], n_lag, 1))
    X_test = X_test.reshape((X_test.shape[0], n_lag, 1))

    # Build the CNN model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Measure training time
    start_time = time.time()

    # Train the model
    model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

    # Record the time taken
    training_time = time.time() - start_time

    # Make predictions
    y_pred = model.predict(X_test)

    # Inverse transform the predicted and test data to original scale
    y_pred_inv = scaler.inverse_transform(y_pred)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate MSE and MAPE
    mse = mean_squared_error(y_test_inv, y_pred_inv)
    mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

    # Store the results
    results.append({
        'Column': col,
        'MSE': mse,
        'MAPE': mape,
        'Training Time (s)': training_time
    })

# Convert the results into a DataFrame and display it
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('ATM_CNN_univariate_scaled_without_tuning.csv', index=False)

# Optionally, download the file (if you're on Google Colab or similar)
from google.colab import files
files.download('ATM_CNN_univariate_scaled_without_tuning.csv')

Training CNN model for column: NN5 - 001


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
Training CNN model for column: NN5 - 002


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Training CNN model for column: NN5 - 003


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Training CNN model for column: NN5 - 111


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Tuned

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import time
from itertools import product

# Assuming the data is already loaded as 'data'
n_test = 30
n_lag = 100  # Rolling window size

# Function to create rolling windows
def create_rolling_window(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

# Placeholder for results
results = []

# Set the hyperparameters for tuning
hidden_layers_options = [1, 2]
filters_options = [64, 128]  # Number of filters for Conv1D layers
kernel_sizes = [2, 3]  # Different kernel sizes for convolution
learning_rates = [0.1, 0.01, 0.001]
batch_sizes = [32, 64]
n_epochs = 100

# Scaling the data using MinMaxScaler (0 to 1 range)
scaler = MinMaxScaler()

data = data.iloc[:, 96:].copy()
# Loop over each column (excluding the 'Date' column)
for col in data.columns:
    print(f'Training CNN model for column: {col}')

    # Prepare the data for the current column
    series = data[col].values.reshape(-1, 1)  # Reshape for scaling (1 feature)

    # Scale the series
    series_scaled = scaler.fit_transform(series)

    # Create rolling windows
    X, y = create_rolling_window(series_scaled, n_lag)

    # Split into train and test
    X_train, X_test = X[:-n_test], X[-n_test:]
    y_train, y_test = y[:-n_test], y[-n_test:]

    # Reshape the data for CNN, needs 3D shape [samples, timesteps, features]
    X_train = X_train.reshape((X_train.shape[0], n_lag, 1))
    X_test = X_test.reshape((X_test.shape[0], n_lag, 1))

    # Iterate over all combinations of hyperparameters
    for num_layers, num_filters, kernel_size, lr, batch_size in product(hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes):

        print(f"Hyperparameters: Layers: {num_layers}, Filters: {num_filters}, Kernel Size: {kernel_size}, LR: {lr}, Batch Size: {batch_size}")

        # Build the CNN model
        model = tf.keras.models.Sequential()

        # Add convolutional layers
        model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', input_shape=(n_lag, 1)))
        model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

        for i in range(1, num_layers):
            model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu'))
            model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

        # Flatten the output from the convolutional layers
        model.add(tf.keras.layers.Flatten())

        # Add the output layer
        model.add(tf.keras.layers.Dense(1))

        # Compile the model with the specified learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mse')

        # Measure training time
        start_time = time.time()

        # Train the model
        model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

        # Record the time taken
        training_time = time.time() - start_time

        # Make predictions
        y_pred = model.predict(X_test)

        # Inverse transform the predicted and test data to original scale
        y_pred_inv = scaler.inverse_transform(y_pred)
        y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Check for NaN values in the predictions or test data
        if np.isnan(y_pred_inv).any() or np.isnan(y_test_inv).any():
            print(f"NaN values detected in column {col} for the hyperparameters: Layers={num_layers}, Filters={num_filters}, Kernel Size={kernel_size}, LR={lr}, Batch Size={batch_size}")
            continue  # Skip this iteration if NaNs are present

        # Calculate MSE and MAPE
        mse = mean_squared_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

        # Store the results
        results.append({
            'Column': col,
            'Layers': num_layers,
            'Filters': num_filters,
            'Kernel Size': kernel_size,
            'Learning Rate': lr,
            'Batch Size': batch_size,
            'MSE': mse,
            'MAPE': mape,
            'Training Time (s)': training_time
        })

# Convert the results into a DataFrame and display it
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file for download
results_df.to_csv('ATM_CNN_univariate_scaled_with_tuning_111.csv', index=False)

# Optionally, download the file (if you're on Google Colab or similar)
from google.colab import files
files.download('ATM_CNN_univariate_scaled_with_tuning_111.csv')

Training CNN model for column: NN5 - 097
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.1, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.1, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.01, Batch Size: 32


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Hyperparameters: Layers: 2, Filters: 128, Kernel Size: 3, LR: 0.001, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install tslearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 14.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import StandardScaler

# Assuming the data is already loaded as 'data'
# Exclude the 'Date' column if present, keep only the series columns for clustering
time_series_data = data.copy()

# Normalize the data for clustering
scaler = StandardScaler()
normalized_data = scaler.fit_transform(time_series_data.T)  # Transpose to have series in rows

# Get the column names (time series identifiers)
column_names = time_series_data.columns.tolist()

# 1. Euclidean Distance Clustering (KMeans)
print("Performing KMeans clustering using Euclidean distance...")

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(normalized_data)
euclidean_labels = kmeans.labels_

# List of column names in each cluster (Euclidean)
euclidean_cluster_1 = [column_names[i] for i in np.where(euclidean_labels == 0)[0].tolist()]
euclidean_cluster_2 = [column_names[i] for i in np.where(euclidean_labels == 1)[0].tolist()]

print(f"Cluster 1 (Euclidean): {euclidean_cluster_1}")
print(f"Cluster 2 (Euclidean): {euclidean_cluster_2}")

# 2. DTW Clustering (TimeSeriesKMeans)
print("Performing DTW-based clustering...")

dtw_kmeans = TimeSeriesKMeans(n_clusters=2, metric="dtw", random_state=42)
dtw_kmeans.fit(time_series_data.T)  # Using transpose to cluster the series
dtw_labels = dtw_kmeans.labels_

# List of column names in each cluster (DTW)
dtw_cluster_1 = [column_names[i] for i in np.where(dtw_labels == 0)[0].tolist()]
dtw_cluster_2 = [column_names[i] for i in np.where(dtw_labels == 1)[0].tolist()]

print(f"Cluster 1 (DTW): {dtw_cluster_1}")
print(f"Cluster 2 (DTW): {dtw_cluster_2}")

Performing KMeans clustering using Euclidean distance...
Cluster 1 (Euclidean): ['NN5 - 002', 'NN5 - 006', 'NN5 - 007', 'NN5 - 009', 'NN5 - 010', 'NN5 - 012', 'NN5 - 013', 'NN5 - 014', 'NN5 - 015', 'NN5 - 020', 'NN5 - 021', 'NN5 - 022', 'NN5 - 023', 'NN5 - 025', 'NN5 - 026', 'NN5 - 027', 'NN5 - 028', 'NN5 - 031', 'NN5 - 033', 'NN5 - 035', 'NN5 - 036', 'NN5 - 037', 'NN5 - 039', 'NN5 - 040', 'NN5 - 041', 'NN5 - 044', 'NN5 - 045', 'NN5 - 046', 'NN5 - 049', 'NN5 - 051', 'NN5 - 053', 'NN5 - 054', 'NN5 - 055', 'NN5 - 056', 'NN5 - 057', 'NN5 - 059', 'NN5 - 060', 'NN5 - 061', 'NN5 - 063', 'NN5 - 064', 'NN5 - 069', 'NN5 - 070', 'NN5 - 071', 'NN5 - 072', 'NN5 - 073', 'NN5 - 074', 'NN5 - 075', 'NN5 - 076', 'NN5 - 077', 'NN5 - 078', 'NN5 - 079', 'NN5 - 081', 'NN5 - 082', 'NN5 - 084', 'NN5 - 085', 'NN5 - 086', 'NN5 - 088', 'NN5 - 089', 'NN5 - 091', 'NN5 - 092', 'NN5 - 094', 'NN5 - 097', 'NN5 - 099', 'NN5 - 100', 'NN5 - 103', 'NN5 - 105', 'NN5 - 106', 'NN5 - 107', 'NN5 - 108', 'NN5 - 111']
Cluster 2

In [ ]:
# Assuming you have already run the clustering and obtained 'euclidean_cluster_1', 'euclidean_cluster_2', 'dtw_cluster_1', and 'dtw_cluster_2'

# Convert the clusters into sets for easier comparison
euclidean_cluster_1_set = set(euclidean_cluster_1)
euclidean_cluster_2_set = set(euclidean_cluster_2)
dtw_cluster_1_set = set(dtw_cluster_1)
dtw_cluster_2_set = set(dtw_cluster_2)

# Compare the clusters: Items that are in both Euclidean Cluster 1 and DTW Cluster 1
common_in_cluster_1 = euclidean_cluster_1_set.intersection(dtw_cluster_1_set)
common_in_cluster_2 = euclidean_cluster_2_set.intersection(dtw_cluster_2_set)

# Compare the clusters: Items that are in Euclidean Cluster 1 and DTW Cluster 2 (and vice versa)
common_between_different_clusters_1 = euclidean_cluster_1_set.intersection(dtw_cluster_2_set)
common_between_different_clusters_2 = euclidean_cluster_2_set.intersection(dtw_cluster_1_set)

# Summary of comparison
print(f"Common items in Euclidean Cluster 1 and DTW Cluster 1: {len(common_in_cluster_1)}")
print(f"Common items in Euclidean Cluster 2 and DTW Cluster 2: {len(common_in_cluster_2)}")
print(f"Common items in Euclidean Cluster 1 and DTW Cluster 2: {len(common_between_different_clusters_1)}")
print(f"Common items in Euclidean Cluster 2 and DTW Cluster 1: {len(common_between_different_clusters_2)}")

Common items in Euclidean Cluster 1 and DTW Cluster 1: 65
Common items in Euclidean Cluster 2 and DTW Cluster 2: 40
Common items in Euclidean Cluster 1 and DTW Cluster 2: 5
Common items in Euclidean Cluster 2 and DTW Cluster 1: 1


# One random

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import random
from itertools import product

# Function to create rolling windows
def create_rolling_window(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

# Scaling function
def scale_data(series):
    scaler = MinMaxScaler()
    return scaler, scaler.fit_transform(series.reshape(-1, 1))

# CNN Training Function with Hyperparameter Tuning
def train_cnn_model_with_tuning(X_train, y_train, X_val, y_val, n_lag, hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes, n_epochs=100):
    best_model = None
    best_mse = float('inf')
    best_hyperparams = None

    # Iterate over all combinations of hyperparameters
    for num_layers, num_filters, kernel_size, lr, batch_size in product(hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes):

        print(f"Hyperparameters: Layers: {num_layers}, Filters: {num_filters}, Kernel Size: {kernel_size}, LR: {lr}, Batch Size: {batch_size}")

        # Build the CNN model
        model = tf.keras.models.Sequential()

        # Add Conv1D layers
        for i in range(num_layers):
            if i == 0:
                model.add(tf.keras.layers.Conv1D(num_filters, kernel_size=kernel_size, activation='relu', input_shape=(n_lag, 1)))
            else:
                model.add(tf.keras.layers.Conv1D(num_filters, kernel_size=kernel_size, activation='relu'))

        # Flatten the output and add the Dense output layer
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(1))

        # Compile the model with the specified learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mse')

        # Train the model
        model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

        # Validate the model to check performance
        y_val_pred = model.predict(X_val)
        mse_val = mean_squared_error(y_val, y_val_pred)

        # Keep track of the best model and hyperparameters
        if mse_val < best_mse:
            best_mse = mse_val
            best_model = model
            best_hyperparams = (num_layers, num_filters, kernel_size, lr, batch_size)

    # Return the best trained model and best hyperparameters
    return best_model, best_hyperparams

# Function to process and predict a cluster
def process_cluster_with_tuning(cluster_data, cluster_name, n_lag=100, n_test=30, hidden_layers_options=[1, 2], filters_options=[64, 128], kernel_sizes=[2, 3], learning_rates=[0.1, 0.01, 0.001], batch_sizes=[32, 64], n_epochs=100):
    # Select a random series from the cluster
    selected_series = random.choice(cluster_data.columns)

    # Print the name of the randomly selected time series
    print(f"Randomly selected time series for {cluster_name}: {selected_series}")

    # Prepare the data for the selected series
    series = cluster_data[selected_series].values
    scaler, series_scaled = scale_data(series)

    # Create rolling windows
    X, y = create_rolling_window(series_scaled, n_lag)

    # Split into train, test, and validation
    X_train, X_val, X_test = X[:-n_test*2], X[-n_test*2:-n_test], X[-n_test:]
    y_train, y_val, y_test = y[:-n_test*2], y[-n_test*2:-n_test], y[-n_test:]

    # Reshape the data for Conv1D [samples, time steps, features]
    X_train = X_train.reshape((X_train.shape[0], n_lag, 1))
    X_val = X_val.reshape((X_val.shape[0], n_lag, 1))
    X_test = X_test.reshape((X_test.shape[0], n_lag, 1))

    # Train the CNN model with hyperparameter tuning
    best_model, best_hyperparams = train_cnn_model_with_tuning(X_train, y_train, X_val, y_val, n_lag, hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes, n_epochs)

    # Predict the rest of the series in the cluster using the best model
    results = []
    for col in cluster_data.columns:
        series_to_predict = cluster_data[col].values
        _, series_scaled_to_predict = scale_data(series_to_predict)
        X_to_predict, y_to_predict = create_rolling_window(series_scaled_to_predict, n_lag)

        # Reshape for Conv1D
        X_to_predict = X_to_predict.reshape((X_to_predict.shape[0], n_lag, 1))

        # Predict using the best model
        y_pred = best_model.predict(X_to_predict)

        # Inverse transform the predicted and test data to original scale
        y_pred_inv = scaler.inverse_transform(y_pred)
        y_test_inv = scaler.inverse_transform(y_to_predict.reshape(-1, 1))

        # Calculate MSE and MAPE
        mse = mean_squared_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

        # Store the results in a list of dictionaries
        results.append({
            'Cluster': cluster_name,
            'Time Series': col,
            'Selected Series for Training': selected_series,
            'MSE': mse,
            'MAPE': mape,
            'Best Hyperparameters': best_hyperparams
        })

    return pd.DataFrame(results)

# Assuming you have already created clusters (euclidean_cluster_1, euclidean_cluster_2, dtw_cluster_1, dtw_cluster_2)
# Process each cluster and train a CNN model for one series in each

# For Euclidean Cluster 1
euclidean_cluster_1_results = process_cluster_with_tuning(data[euclidean_cluster_1], "Euclidean Cluster 1")

# For Euclidean Cluster 2
euclidean_cluster_2_results = process_cluster_with_tuning(data[euclidean_cluster_2], "Euclidean Cluster 2")

# For DTW Cluster 1
dtw_cluster_1_results = process_cluster_with_tuning(data[dtw_cluster_1], "DTW Cluster 1")

# For DTW Cluster 2
dtw_cluster_2_results = process_cluster_with_tuning(data[dtw_cluster_2], "DTW Cluster 2")

# Combine all results into one DataFrame
all_results = pd.concat([euclidean_cluster_1_results, euclidean_cluster_2_results, dtw_cluster_1_results, dtw_cluster_2_results])

# # Save the combined results to a CSV file
# all_results.to_csv('clustered_cnn_results.csv', index=False)

# # Show the results
# print(all_results)

Randomly selected time series for Euclidean Cluster 1: NN5 - 076
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.1, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.1, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.01, Batch Size: 32


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
Hyperparameters: Layers: 2, Filters: 128, Kernel Size: 3, LR: 0.001, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Hyperparameters: Layers: 2, Filters: 128, Kernel Size: 3, LR: 0.001, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2

# Multivariate

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import random
from itertools import product
import time

# Function to create rolling windows for multivariate input
def create_multivariate_rolling_window(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])  # Shape: (window_size, num_features)
        y.append(data[i + window_size, target_index])
    return np.array(X), np.array(y)

# Scaling function for multivariate data
def scale_multivariate_data(data):
    scaler = MinMaxScaler()
    return scaler, scaler.fit_transform(data)

# Multivariate CNN Training Function with Hyperparameter Tuning
def train_multivariate_cnn_with_tuning(X_train, y_train, X_val, y_val, n_lag, hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes, n_epochs=100):
    best_model = None
    best_mse = float('inf')
    best_hyperparams = None
    training_times_per_setting = []

    # Iterate over all combinations of hyperparameters
    for num_layers, num_filters, kernel_size, lr, batch_size in product(hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes):
        print(f"Hyperparameters: Layers: {num_layers}, Filters: {num_filters}, Kernel Size: {kernel_size}, LR: {lr}, Batch Size: {batch_size}")

        # Build the CNN model
        model = tf.keras.models.Sequential()

        # Add Conv1D layers
        for i in range(num_layers):
            if i == 0:
                # First Conv1D layer with input shape
                model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', input_shape=(n_lag, X_train.shape[2])))
            else:
                model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu'))

        # Add a Flatten layer to connect Conv1D layers to Dense output layer
        model.add(tf.keras.layers.Flatten())

        # Add the output layer
        model.add(tf.keras.layers.Dense(1))

        # Compile the model with the specified learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mse')

        # Track the start time
        start_time = time.time()

        # Train the model
        model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)

        # Track the end time and calculate training duration
        end_time = time.time()
        training_duration = end_time - start_time
        training_times_per_setting.append((num_layers, num_filters, kernel_size, lr, batch_size, training_duration))

        print(f"Training time for current setting: {training_duration:.2f} seconds")

        # Validate the model to check performance
        y_val_pred = model.predict(X_val)
        mse_val = mean_squared_error(y_val, y_val_pred)

        # Keep track of the best model and hyperparameters
        if mse_val < best_mse:
            best_mse = mse_val
            best_model = model
            best_hyperparams = (num_layers, num_filters, kernel_size, lr, batch_size)

    # Return the best trained model, best hyperparameters, and individual training times
    return best_model, best_hyperparams, training_times_per_setting

# Function to process a cluster, tune on one series, and apply the model to all series
def process_cluster_with_tuning(cluster_data, cluster_name, n_lag=100, n_test=30, hidden_layers_options=[1, 2], filters_options=[64, 128], kernel_sizes=[2, 3], learning_rates=[0.1, 0.01, 0.001], batch_sizes=[32, 64], n_epochs=100):
    results = []

    # Select a random series from the cluster for tuning
    selected_series = random.choice(cluster_data.columns)
    target_index = cluster_data.columns.get_loc(selected_series)
    print(f"Tuning model for target series {selected_series} in cluster {cluster_name}")

    # Scale the data for all series in the cluster
    scaler, scaled_data = scale_multivariate_data(cluster_data.values)

    # Create rolling windows with multivariate data, using selected series as target
    X, y = create_multivariate_rolling_window(scaled_data, n_lag, target_index)

    # Split into train, validation, and test sets
    X_train, X_val, X_test = X[:-n_test*2], X[-n_test*2:-n_test], X[-n_test:]
    y_train, y_val, y_test = y[:-n_test*2], y[-n_test*2:-n_test], y[-n_test:]

    # Train the multivariate CNN model with hyperparameter tuning on the selected series
    best_model, best_hyperparams, training_times_per_setting = train_multivariate_cnn_with_tuning(X_train, y_train, X_val, y_val, n_lag, hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes, n_epochs)

    # Display individual training times per setting
    for (num_layers, num_filters, kernel_size, lr, batch_size, duration) in training_times_per_setting:
        print(f"Hyperparameters: Layers={num_layers}, Filters={num_filters}, Kernel Size={kernel_size}, LR={lr}, Batch Size={batch_size} -> Training Time: {duration:.2f} seconds")

    # Apply the tuned model to forecast each series in the cluster
    for target_series in cluster_data.columns:
        target_index = cluster_data.columns.get_loc(target_series)
        print(f"Applying tuned model to forecast series {target_series} in cluster {cluster_name}")

        # Create rolling windows for the current target series
        X_all, y_all = create_multivariate_rolling_window(scaled_data, n_lag, target_index)

        # Use the test portion
        X_test, y_test = X_all[-n_test:], y_all[-n_test:]

        # Predict using the tuned model
        y_pred = best_model.predict(X_test)

        # Inverse transform the predicted and test data to the original scale for the target series
        y_pred_full = np.zeros((y_pred.shape[0], scaled_data.shape[1]))  # Zero-padding for inverse transform
        y_pred_full[:, target_index] = y_pred.flatten()
        y_pred_inv = scaler.inverse_transform(y_pred_full)[:, target_index]

        y_test_full = np.zeros((y_test.shape[0], scaled_data.shape[1]))
        y_test_full[:, target_index] = y_test.flatten()
        y_test_inv = scaler.inverse_transform(y_test_full)[:, target_index]

        # Calculate MSE and MAPE for the target series
        mse = mean_squared_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

        # Append the results
        results.append({
            'Cluster': cluster_name,
            'Time Series': target_series,
            'MSE': mse,
            'MAPE': mape,
            'Best Hyperparameters': best_hyperparams,
            'Training Times (s) per Setting': training_times_per_setting
        })

    return pd.DataFrame(results)

# For Euclidean Cluster 1
euclidean_cluster_1_results = process_cluster_with_tuning(data[euclidean_cluster_1], "Euclidean Cluster 1")

# For Euclidean Cluster 2
euclidean_cluster_2_results = process_cluster_with_tuning(data[euclidean_cluster_2], "Euclidean Cluster 2")

# For DTW Cluster 1
dtw_cluster_1_results = process_cluster_with_tuning(data[dtw_cluster_1], "DTW Cluster 1")

# For DTW Cluster 2
dtw_cluster_2_results = process_cluster_with_tuning(data[dtw_cluster_2], "DTW Cluster 2")

# Combine all results into one DataFrame
all_results = pd.concat([euclidean_cluster_1_results, euclidean_cluster_2_results, dtw_cluster_1_results, dtw_cluster_2_results])

# Save or display results
print(all_results)


Tuning model for target series NN5 - 028 in cluster Euclidean Cluster 1
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.1, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training time for current setting: 5.80 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.1, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training time for current setting: 5.01 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.01, Batch Size: 32


...

Training time for current setting: 6.25 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Hyperparameters: Layers: 2, Filters: 128, Kernel Size: 3, LR: 0.001, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training time for current setting: 7.69 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Hyperparameters: Layers: 2, Filters: 128, Kernel Size: 3, LR: 0.001, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training time for current setting: 6.32 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Hyperparameters: Layers=1, Filters=64, Kernel Size=2, LR=0.1, Batch Size=32 -> Training Time: 5.39 seconds
Hyperparameters: Layers=1, Filters=64, Kernel Size=2, LR=0.1, Batch Size=64 -> Training Time: 4.46 seconds
Hyperparameters: Layers=1, Filters=64, Kernel Size=2, LR=0.01, Batch Size=32 -> Training Time: 5.49 seconds
Hyperparameters: Layers=1, Filters=64, Kernel Size=2, LR=0.01, Batch Size=64 -> Training Time: 4.42 seconds
Hyperparameters: Layers=1, Filters=64, Kernel Size=2, LR=0.001, Batch Size=32 -> Training Time: 5.47 seconds
Hyperparameters: Layers=1, Filters=64, Kernel Size=2, LR=0.001, Batch Size=64 -> Training Time: 4.39 seconds
Hyperparameters: Layers=1, Filters=64, Kernel Size=3, LR=0.1, Batch Size=32 -> Training Time: 5.56 seconds
Hyperparameters: Layers=1, Filters=64, Kernel Size=3, LR=0.1, Batch Size=64 -> Training Time: 4.47 seconds
Hyperparameters: Layers=1, Filters=64, Kernel Size=

# Create the average of a cluster and then tune it and apply it for whole cluster

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import random
from itertools import product
import time

# Scaling function for data
def scale_data(data):
    scaler = MinMaxScaler()
    return scaler, scaler.fit_transform(data)

# CNN Training Function with Hyperparameter Tuning
def train_cnn_with_tuning(X_train, y_train, X_val, y_val, n_lag, hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes, n_epochs=100):
    best_model = None
    best_mse = float('inf')
    best_hyperparams = None
    training_times_per_setting = []

    for num_layers, num_filters, kernel_size, lr, batch_size in product(hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes):
        print(f"Hyperparameters: Layers: {num_layers}, Filters: {num_filters}, Kernel Size: {kernel_size}, LR: {lr}, Batch Size: {batch_size}")

        # Build the CNN model
        model = tf.keras.models.Sequential()
        for i in range(num_layers):
            if i == 0:
                model.add(tf.keras.layers.Conv1D(num_filters, kernel_size=kernel_size, activation='relu', input_shape=(n_lag, 1)))
            else:
                model.add(tf.keras.layers.Conv1D(num_filters, kernel_size=kernel_size, activation='relu'))
            model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(1))

        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mse')

        start_time = time.time()
        model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0)
        training_duration = time.time() - start_time
        training_times_per_setting.append((num_layers, num_filters, kernel_size, lr, batch_size, training_duration))

        y_val_pred = model.predict(X_val)
        mse_val = mean_squared_error(y_val, y_val_pred)

        if mse_val < best_mse:
            best_mse = mse_val
            best_model = model
            best_hyperparams = (num_layers, num_filters, kernel_size, lr, batch_size)

    return best_model, best_hyperparams, training_times_per_setting

# Function to create rolling windows for univariate input
def create_rolling_window(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

# Function to process a cluster, tune on one series, and apply the model to all series
def process_cluster_with_cnn_tuning(cluster_data, cluster_name, n_lag=100, n_test=30, hidden_layers_options=[1, 2], filters_options=[64, 128], kernel_sizes=[2, 3], learning_rates=[0.1, 0.01, 0.001], batch_sizes=[32, 64], n_epochs=100):
    results = []

    # Create an average series for tuning
    average_series = cluster_data.mean(axis=1).values
    scaler, scaled_data = scale_data(average_series.reshape(-1, 1))
    X, y = create_rolling_window(scaled_data, n_lag)

    X_train, X_val, X_test = X[:-n_test*2], X[-n_test*2:-n_test], X[-n_test:]
    y_train, y_val, y_test = y[:-n_test*2], y[-n_test*2:-n_test], y[-n_test:]

    # Train CNN with hyperparameter tuning
    best_model, best_hyperparams, training_times_per_setting = train_cnn_with_tuning(
        X_train, y_train, X_val, y_val, n_lag, hidden_layers_options, filters_options, kernel_sizes, learning_rates, batch_sizes, n_epochs
    )

    # Apply the tuned model to each series in the cluster
    for target_series in cluster_data.columns:
        print(f"Applying tuned model to forecast series {target_series} in cluster {cluster_name}")
        series_data = cluster_data[target_series].values.reshape(-1, 1)
        _, scaled_series = scale_data(series_data)
        X_all, y_all = create_rolling_window(scaled_series, n_lag)

        X_test, y_test = X_all[-n_test:], y_all[-n_test:]
        y_pred = best_model.predict(X_test)

        # Inverse transform predictions
        y_pred_full = np.zeros((y_pred.shape[0], 1))
        y_pred_full[:, 0] = y_pred.flatten()
        y_pred_inv = scaler.inverse_transform(y_pred_full)[:, 0]

        y_test_full = np.zeros((y_test.shape[0], 1))
        y_test_full[:, 0] = y_test.flatten()
        y_test_inv = scaler.inverse_transform(y_test_full)[:, 0]

        mse = mean_squared_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

        results.append({
            'Cluster': cluster_name,
            'Time Series': target_series,
            'MSE': mse,
            'MAPE': mape,
            'Best Hyperparameters': best_hyperparams,
            'Training Times (s) per Setting': training_times_per_setting
        })

    return pd.DataFrame(results)

# Example usage for clusters
euclidean_cluster_1_results = process_cluster_with_cnn_tuning(data[euclidean_cluster_1], "Euclidean Cluster 1")
euclidean_cluster_2_results = process_cluster_with_cnn_tuning(data[euclidean_cluster_2], "Euclidean Cluster 2")
dtw_cluster_1_results = process_cluster_with_cnn_tuning(data[dtw_cluster_1], "DTW Cluster 1")
dtw_cluster_2_results = process_cluster_with_cnn_tuning(data[dtw_cluster_2], "DTW Cluster 2")

# Combine all results into one DataFrame
all_results = pd.concat([euclidean_cluster_1_results, euclidean_cluster_2_results, dtw_cluster_1_results, dtw_cluster_2_results])
print(all_results)


Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.1, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.1, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Hyperparameters: Layers: 1, Filters: 64, Kernel Size: 2, LR: 0.01, Batch Size: 32


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
Hyperparameters: Layers: 2, Filters: 128, Kernel Size: 3, LR: 0.001, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
Hyperparameters: Layers: 2, Filters: 128, Kernel Size: 3, LR: 0.001, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Applying tuned model to forecast series NN5 - 001 in cluster DTW Cluster 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Applying tuned model to forecast series NN5 - 003 in cluster DTW Cluster 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Applying tuned model to forecast series NN5 - 004 in cluster DTW Cluster 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Applying tuned model to forecast series NN5 - 005 in cluster DTW Cluster 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Applying tuned model to forecast series NN5 - 006 in cluster DTW Cluster 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Applying tuned model to forecast series NN5 - 008 in cluster DTW Cluster 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Applying tuned model to forecast series NN5 - 011 in cluster DTW Cluster 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Applying tuned model to forecast series NN5 - 013 in cluster DTW Cluster 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Applying tuned model to forecast series NN5 - 016 in clus